In [1]:
# Importing the necessary packages
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# Reading in each csv file for the years of customer data
data = pd.read_csv('data (1) copy.csv', encoding = 'ISO-8859-1')

In [3]:
# Spliting the Created column into two rows with the date and time as seperate columns
data['date_created'], data['time_created'] = data['Created'].str.split(' ', 1).str

In [4]:
# Printing out the first 5 rows of each column in the dataset
print(data.head())

   #Prod                    Agent Label +ID        Created           ID  \
0    1.0      Church Bulletin 0011 (109398)  7/30/14 14:36  657917124.0   
1    1.0  Benefits and More 4 Paid (143595)  10/5/15 19:15  658935850.0   
2    1.0  Benefits and More 4 Paid (143595)   5/2/16 20:16  659370939.0   
3    1.0  Benefits and More 4 Paid (143595)  4/22/16 19:55  659356995.0   
4    1.0       Benefits and More 4 (116063)  7/28/16 16:01  659940549.0   

    Income  Last Pay Amount  Last Pay Complete Last Pay Date  Last Pay Status  \
0  36250.0            34.95                1.0       8/25/14              1.0   
1  36250.0            29.95                1.0       10/5/15              1.0   
2  36250.0            29.95                1.0        5/2/16              1.0   
3  36250.0            39.95                1.0       5/12/16              1.0   
4  36250.0              NaN                NaN           NaN              NaN   

  Last Pay Type  Pay Method State              Status  ZipCode

In [5]:
# Converting the two new split columns into datetime objects
data['date_created'] = pd.to_datetime(data['date_created'])
data['Last Pay Date'] = pd.to_datetime(data['Last Pay Date'])

# Taking the difference between the last pay date and the date the account was created 
# Using the dt.days function to calculate amount of days in between those dates
data['days'] = (data['Last Pay Date'] - data['date_created']).dt.days

# Dividing the number of days by 30 to get the number of months with the company
data['months'] = data['days']/30

In [6]:
# Extracting the year from the date_created column to create a 'year' column
data['year'] = pd.DatetimeIndex(data['date_created']).year

In [7]:
# Rounding up the 'months' and 'days' columns to the hundredth decimal place
data['months'] = np.round(data['months'], 2)
data['days'] = np.round(data['days'], 2)

## **Changing Column Types and Names**

In [8]:
# Showing the data types for each column
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119833 entries, 0 to 119832
Data columns (total 20 columns):
#Prod                119794 non-null float64
Agent Label +ID      119832 non-null object
Created              119832 non-null object
ID                   119832 non-null float64
Income               118524 non-null float64
Last Pay Amount      113509 non-null float64
Last Pay Complete    113509 non-null float64
Last Pay Date        113509 non-null datetime64[ns]
Last Pay Status      113509 non-null float64
Last Pay Type        113288 non-null object
Pay Method           118063 non-null object
State                119832 non-null object
Status               119832 non-null object
ZipCodeNew           119832 non-null float64
Stage                100142 non-null object
date_created         119832 non-null datetime64[ns]
time_created         119832 non-null object
days                 113509 non-null float64
months               113509 non-null float64
year                 119832 

In [9]:
# Changing data column types to their appropriate types
data['ID'] = data['ID'].astype(str)
data['ZipCodeNew'] = data['ZipCodeNew'].astype(str)

In [10]:
# Deleting the row 'Created' as it's redundant
data.drop(['Created'], axis=1)

# Changing the order of the columns in the dataset
data = data[['ID', 'year', 'Agent Label +ID', 'date_created', 'time_created', 'Last Pay Date', 'Pay Method', 'State', 'ZipCodeNew',
             'Last Pay Amount', 'Last Pay Complete', 'Last Pay Status', 'Last Pay Type', '#Prod', 'Stage',
             'Status', 'Income', 'days', 'months']]

In [11]:
# Renaming the columns in the data.csv dataset
data.columns = ['id', 'year', 'agent', 'date_created', 'time_created', 'lp_date', 'pay_method', 'state', 'zip_code',
               'lp_amount', 'lp_complete', 'lp_status', 'lp_type', 'prod', 'stage', 'status', 'income', 'days', 'months']

## **Imputations**

In [12]:
# Same method we used in Data Mining to calculate number of missing values per column

# Creating an empty list for column names
names = []

# Creating an empty list for the number of null values in each column
values = []

# Checking for Missing Values
for col in data.columns:
    names.append(col)
    values.append(data[col].isnull().sum())
    print(names[-1],values[-1])

id 0
year 1
agent 1
date_created 1
time_created 1
lp_date 6324
pay_method 1770
state 1
zip_code 0
lp_amount 6324
lp_complete 6324
lp_status 6324
lp_type 6545
prod 39
stage 19691
status 1
income 1309
days 6324
months 6324


In [13]:
# Imputing the income column using the median of the income column

data['income'] = data['income'].fillna(data['income'].median())
data['income'] = data['income'].replace(0, data['income'].median())
data['income'].head()

0    36250.0
1    36250.0
2    36250.0
3    36250.0
4    36250.0
Name: income, dtype: float64

In [14]:
# Deleting rows with no pay method given since most of these rows have many missing values other than pay method

data = data[pd.notnull(data['pay_method'])] 

# Deleting rows that do not include a Last Pay Date since the customer never paid for a product
data = data[pd.notnull(data['lp_date'])]

# Deleting one row where the zip code was never given
data = data[pd.notnull(data['zip_code'])]

# Deleting rows with null lp_type values since no payment was received so no type was recorded
data = data[pd.notnull(data['lp_type'])]

# Deleting rows that include Quick Cancel because these customer canceled the order before the end of a full month
data = data[~data['stage'].isin(['Quick Cancel'])]

# Deleting rows that have 0 months stayed since these customers never paid for the service so they are not useful
data = data[~data['months'].isin([0])]

data.head()

,id,year,agent,date_created,time_created,lp_date,pay_method,state,zip_code,lp_amount,lp_complete,lp_status,lp_type,prod,stage,status,income,days,months
0,657917124.0,2014.0,Church Bulletin 0011 (109398),2014-07-30,14:36,2014-08-25,Discover,MA,1001.0,34.95,1.0,1.0,Refund,1.0,Back in Inventory,Returned <30 days,36250.0,26.0,0.87
3,659356995.0,2016.0,Benefits and More 4 Paid (143595),2016-04-22,19:55,2016-05-12,MasterCard,MA,1001.0,39.95,1.0,1.0,Refund,1.0,Back in Inventory,Returned <30 days,36250.0,20.0,0.67
5,657847231.0,2014.0,Newspaper (109455),2014-06-16,15:40,2014-07-03,Visa,MA,1002.0,34.95,1.0,0.0,Sale,1.0,NaN,Decline Cancel,74083.0,17.0,0.57
7,672438569.0,2017.0,Yasha - Pers1 (264925),2017-10-20,15:02,2018-02-09,MasterCard,MA,1002.0,79.90,1.0,0.0,Sale,1.0,Lost/Not Paid,Returned,74083.0,112.0,3.73
9,657897142.0,2014.0,Angies List - Forte (109439),2014-07-18,13:48,2014-08-05,MasterCard,MA,1005.0,34.95,1.0,0.0,Sale,1.0,NaN,Decline Cancel,44836.0,18.0,0.60


In [15]:
# Binning categorical columns to larger groups
# Printing unique values for columns
print(data['pay_method'].unique())

#Binning 'pay_method' into 4 groups: Credit card, ACH, Paper Check, & Other
data['pay_method'] = data['pay_method'].replace({'Visa': 'credit card', 'MasterCard': 'credit card', 
                                                 'Discover': 'credit card', 'American Express': 'credit card',
                                                'Mastercard': 'credit card', 'Maestro': 'credit card',
                                                'Paper_Check': 'paper check'})  

# Counting the number of instances for each value in the 'pay_method' column
print(data['pay_method'].value_counts())

['Discover' 'MasterCard' 'Visa' 'ACH' 'American Express' 'Paper_Check'
 'Other' 'Mastercard' 'Maestro']
credit card    74253
ACH            11028
paper check     1429
Other            143
Name: pay_method, dtype: int64


In [16]:
# Same method we used in Data Mining to calculate number of missing values per column

# Creating an empty list for column names
names = []

# Creating an empty list for the number of null values in each column
values = []

# Checking for Missing Values
for col in data.columns:
    names.append(col)
    values.append(data[col].isnull().sum())
    print(names[-1],values[-1])

id 0
year 0
agent 0
date_created 0
time_created 0
lp_date 0
pay_method 0
state 0
zip_code 0
lp_amount 0
lp_complete 0
lp_status 0
lp_type 0
prod 0
stage 11101
status 0
income 0
days 0
months 0


In [17]:
# Creating a binary code for whether a customer staying over 18 months or not
# 1 = greater than 18 months
data['18_months'] = np.where(data['months'] >= 18., 1, 0)

In [18]:
# Deleting customers for the years 2018 and 2019 since they have not reached the 18 month mark yet
data = data[~((data['status'] == 'Active') & (data['year'] == 2018))]
data = data[~((data['status'] == 'Active') & (data['year'] == 2019))]

In [19]:
# Printing out the unique values and value counts to see what needs to be combined
print(data['status'].unique())

# Changing names of categories into bins
data['status'] = data['status'].replace({'Returned <30 days': 'Returned < 90 days', 'Decline Cancel': 'Decline',
                                        'Order Canceled': 'Order Cancelled', 'Daily Decline': 'Decline', 
                                        'Returned <60 Days': 'Returned < 90 days', 'Chargeback Received': 'Other',
                                        'In Process': 'Other', 'Post Date': 'Other', 'Suspend': 'Other',
                                        'Test': 'Other'}) 

['Returned <30 days' 'Decline Cancel' 'Returned' 'Returned < 90 days'
 'Duplicate' 'Active' 'Order Canceled' 'Decline' 'Returned to Sender'
 'Deactivated' 'Restricted' 'Daily Decline' 'Switched to LW'
 'Returned <60 Days' 'Suspend' 'In Process' 'Collections'
 'Chargeback Received' 'Test' 'Post Date']


In [20]:
# Using the method of one-hot encoding for the different forms of pay methods
data = pd.concat([data, pd.get_dummies(data['pay_method'], prefix = 'pay_method')], axis = 1)
data.head()

,id,year,agent,date_created,time_created,lp_date,pay_method,state,zip_code,lp_amount,...,stage,status,income,days,months,18_months,pay_method_ACH,pay_method_Other,pay_method_credit card,pay_method_paper check
0,657917124.0,2014.0,Church Bulletin 0011 (109398),2014-07-30,14:36,2014-08-25,credit card,MA,1001.0,34.95,...,Back in Inventory,Returned < 90 days,36250.0,26.0,0.87,0,0,0,1,0
3,659356995.0,2016.0,Benefits and More 4 Paid (143595),2016-04-22,19:55,2016-05-12,credit card,MA,1001.0,39.95,...,Back in Inventory,Returned < 90 days,36250.0,20.0,0.67,0,0,0,1,0
5,657847231.0,2014.0,Newspaper (109455),2014-06-16,15:40,2014-07-03,credit card,MA,1002.0,34.95,...,NaN,Decline,74083.0,17.0,0.57,0,0,0,1,0
7,672438569.0,2017.0,Yasha - Pers1 (264925),2017-10-20,15:02,2018-02-09,credit card,MA,1002.0,79.90,...,Lost/Not Paid,Returned,74083.0,112.0,3.73,0,0,0,1,0
9,657897142.0,2014.0,Angies List - Forte (109439),2014-07-18,13:48,2014-08-05,credit card,MA,1005.0,34.95,...,NaN,Decline,44836.0,18.0,0.60,0,0,0,1,0


In [21]:
# Using the method of one-hot encoding for the different forms of pay methods
data = pd.concat([data, pd.get_dummies(data['status'], prefix = 'status')], axis = 1)
data.head()

,id,year,agent,date_created,time_created,lp_date,pay_method,state,zip_code,lp_amount,...,status_Deactivated,status_Decline,status_Duplicate,status_Order Cancelled,status_Other,status_Restricted,status_Returned,status_Returned < 90 days,status_Returned to Sender,status_Switched to LW
0,657917124.0,2014.0,Church Bulletin 0011 (109398),2014-07-30,14:36,2014-08-25,credit card,MA,1001.0,34.95,...,0,0,0,0,0,0,0,1,0,0
3,659356995.0,2016.0,Benefits and More 4 Paid (143595),2016-04-22,19:55,2016-05-12,credit card,MA,1001.0,39.95,...,0,0,0,0,0,0,0,1,0,0
5,657847231.0,2014.0,Newspaper (109455),2014-06-16,15:40,2014-07-03,credit card,MA,1002.0,34.95,...,0,1,0,0,0,0,0,0,0,0
7,672438569.0,2017.0,Yasha - Pers1 (264925),2017-10-20,15:02,2018-02-09,credit card,MA,1002.0,79.90,...,0,0,0,0,0,0,1,0,0,0
9,657897142.0,2014.0,Angies List - Forte (109439),2014-07-18,13:48,2014-08-05,credit card,MA,1005.0,34.95,...,0,1,0,0,0,0,0,0,0,0


In [22]:
# Using the method of one-hot encoding for the different forms of pay methods
data = pd.concat([data, pd.get_dummies(data['state'], prefix = 'state')], axis = 1)
data.head()

,id,year,agent,date_created,time_created,lp_date,pay_method,state,zip_code,lp_amount,...,state_SD,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY
0,657917124.0,2014.0,Church Bulletin 0011 (109398),2014-07-30,14:36,2014-08-25,credit card,MA,1001.0,34.95,...,0,0,0,0,0,0,0,0,0,0
3,659356995.0,2016.0,Benefits and More 4 Paid (143595),2016-04-22,19:55,2016-05-12,credit card,MA,1001.0,39.95,...,0,0,0,0,0,0,0,0,0,0
5,657847231.0,2014.0,Newspaper (109455),2014-06-16,15:40,2014-07-03,credit card,MA,1002.0,34.95,...,0,0,0,0,0,0,0,0,0,0
7,672438569.0,2017.0,Yasha - Pers1 (264925),2017-10-20,15:02,2018-02-09,credit card,MA,1002.0,79.90,...,0,0,0,0,0,0,0,0,0,0
9,657897142.0,2014.0,Angies List - Forte (109439),2014-07-18,13:48,2014-08-05,credit card,MA,1005.0,34.95,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# Printing the counts of 'state' and 'status' to see what dummy variables need to be dropped
print(data.state.value_counts())
print(data.status.value_counts())

TX    7473
CA    7275
FL    6048
NY    5271
NC    4268
GA    3747
OH    3567
PA    3473
MI    3294
IL    2664
VA    2396
TN    2388
LA    2309
AL    2276
SC    2248
NJ    2009
MO    1818
MD    1692
KY    1548
OK    1490
WA    1428
AR    1309
AZ    1271
MA    1210
MS    1157
CO     884
OR     883
WI     870
MN     822
CT     783
NV     670
WV     597
KS     587
NM     568
IA     555
DC     460
ME     422
UT     383
NE     366
DE     290
NH     283
MT     271
ID     249
RI     227
VT     174
IN     157
SD     149
ND     144
WY     143
HI      45
AK      36
NT       1
Name: state, dtype: int64
Returned              30496
Returned < 90 days    20692
Decline               18118
Order Cancelled        6192
Active                 3292
Duplicate              2130
Deactivated            1755
Returned to Sender      921
Switched to LW          742
Restricted              223
Collections              44
Other                    43
Name: status, dtype: int64


In [24]:
# Dropping columns of states and status types that do not have enough answers to be useful in the analysis
data = data.drop(['state_NV', 'state_WV', 'state_KS', 'state_NM', 'state_IA', 'state_DC', 'state_ME', 'state_UT', 
          'state_NE', 'state_DE', 'state_NH', 'state_MT', 'state_ID', 'state_RI', 'state_VT', 'state_IN',
          'state_SD', 'state_ND', 'state_WY', 'state_HI', 'state_AK', 'state_NT', 'status_Other', 
                  'status_Restricted', 'status_Collections'], axis = 1)

data.head()

,id,year,agent,date_created,time_created,lp_date,pay_method,state,zip_code,lp_amount,...,state_OH,state_OK,state_OR,state_PA,state_SC,state_TN,state_TX,state_VA,state_WA,state_WI
0,657917124.0,2014.0,Church Bulletin 0011 (109398),2014-07-30,14:36,2014-08-25,credit card,MA,1001.0,34.95,...,0,0,0,0,0,0,0,0,0,0
3,659356995.0,2016.0,Benefits and More 4 Paid (143595),2016-04-22,19:55,2016-05-12,credit card,MA,1001.0,39.95,...,0,0,0,0,0,0,0,0,0,0
5,657847231.0,2014.0,Newspaper (109455),2014-06-16,15:40,2014-07-03,credit card,MA,1002.0,34.95,...,0,0,0,0,0,0,0,0,0,0
7,672438569.0,2017.0,Yasha - Pers1 (264925),2017-10-20,15:02,2018-02-09,credit card,MA,1002.0,79.90,...,0,0,0,0,0,0,0,0,0,0
9,657897142.0,2014.0,Angies List - Forte (109439),2014-07-18,13:48,2014-08-05,credit card,MA,1005.0,34.95,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# dropping columns we will not be using in the models
data = data.drop(['id', 'agent', 'date_created', 'time_created', 'lp_date', 'pay_method', 'state', 'zip_code',
                 'lp_amount', 'days', 'lp_complete', 'lp_status', 'lp_type', 'prod', 'stage', 'status'], axis = 1)
data.columns

Index(['year', 'income', 'months', '18_months', 'pay_method_ACH',
       'pay_method_Other', 'pay_method_credit card', 'pay_method_paper check',
       'status_Active', 'status_Deactivated', 'status_Decline',
       'status_Duplicate', 'status_Order Cancelled', 'status_Returned',
       'status_Returned < 90 days', 'status_Returned to Sender',
       'status_Switched to LW', 'state_AL', 'state_AR', 'state_AZ', 'state_CA',
       'state_CO', 'state_CT', 'state_FL', 'state_GA', 'state_IL', 'state_KY',
       'state_LA', 'state_MA', 'state_MD', 'state_MI', 'state_MN', 'state_MO',
       'state_MS', 'state_NC', 'state_NJ', 'state_NY', 'state_OH', 'state_OK',
       'state_OR', 'state_PA', 'state_SC', 'state_TN', 'state_TX', 'state_VA',
       'state_WA', 'state_WI'],
      dtype='object')

In [26]:
data.shape

(84648, 47)

In [72]:
# Saving the imputed data as impute_data.csv to begin normalization
data1.to_csv('imputed_data.csv',',')